In [1]:
import pandas as pd
import polars as pl

import numpy as np

import lectura_archivos as fr

# Analisis de Cantidad de Traqueostomias por Servicio de Salud

- En este apartado se quiere ver la cantidad de traqueostomias que se hicieron por Servicio de Salud.
Este analisis se realiza por anio.

## Carga de Archivos de GRD

In [6]:
dfp = fr.leer_grd()


In [7]:
df = dfp.to_pandas()


## Analisis de Traqueostomia

In [5]:
procedimientos = df.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"]
contiene_nuestros_proc = procedimientos.isin(["31.1", "31.29"]).sum(axis=1).astype(bool)
proc_nacionales = df[contiene_nuestros_proc]


In [6]:
metricas = (
    proc_nacionales.groupby(["ANIO_EGRESO", "SERVICIO_SALUD"])
    .agg(
        n_egresos=("COD_HOSPITAL", "count"),
        peso_medio=("IR_29301_PESO", "mean"),
        estancia_media=("ESTANCIA", "mean"),
    )
    .sort_values(["ANIO_EGRESO", "n_egresos"], ascending=False)
)

In [7]:
metricas.to_excel("output/resumen_traqueostomia_hist.xlsx")

# Analisis de Prestaciones por Diagnostico

- En este apartado se quiere ver la cantidad de prestaciones que se realizan por cada diagnostico
principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|

## Analisis Exploratorio

### Obtener conteo de diagnosticos en el Torax

In [8]:
torax = dfp.filter(pl.col("COD_HOSPITAL") == 112103)
torax.groupby(["DIAGNOSTICO1"]).agg(pl.count()).sort(by="count", descending=True)


DIAGNOSTICO1,count
str,u32
"""I25.1""",1366
"""Z51.1""",811
"""U07.1""",706
"""Z51.4""",595
"""Z13.6""",489
"""Z29.1""",394
"""C34.1""",367
"""I35.0""",365
"""Z45.0""",322


- De estos resultados, se puede ver que el diagnostico Z51.4 es el diagnostico mas frecuente
entre 2019 y 2021. Tuvo 1366 egresos.

- A modo de ejemplo, se quiere ver el dataframe completo del diagnostico Z51.4, y analizar
los procedimientos que tiene incluidos.

### Analisis de conteo de procedimientos

### Forma 1

In [9]:
from functools import reduce

i251 = torax.filter(pl.col("DIAGNOSTICO1") == "I25.1").to_pandas()
listas_procedimientos = (
    i251.loc[:, "PROCEDIMIENTO1":"PROCEDIMIENTO30"].fillna("").agg("-".join, axis=1).str.split("-")
)


sumado = reduce(lambda x, y: x + y, listas_procedimientos.values)

### Forma 2

In [10]:
cambiada = pd.melt(
    i251, id_vars="DIAGNOSTICO1", value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)]
)
cambiada["value"].value_counts()


99.29    1196
99.19    1125
89.54    1035
93.96     930
37.22     806
         ... 
78.61       1
39.98       1
99.10       1
37.99       1
37.87       1
Name: value, Length: 257, dtype: int64

## Analisis Completo

- Se quiere ver la cantida dde procedimientos hechos por posicion de procedimiento. O sea,
en el "PROCEDIMIENTO1" que procedimientos se ingresaron, y cuales son sus frecuencias ordenadas
de mayor a menor?

- Tambien se quiere ver la frecuencia global. O sea, para el diagnostico X, cuales son los
procedimientos que mas se le realizan, independiente de la posicion del procedimiento.

#### Cambio de nombre de columnas PROCEDIMIENTO

- Esto se realiza, ya que permite ordenar estas columnas al utilizar .value_counts() en la
proxima celda.

In [13]:
import string

procedimientos = [f"PROCEDIMIENTO{i}" for i in range(1, 31)]
letras = list(string.ascii_uppercase) + ["a", "b", "c", "d"]
codificacion = dict(zip(procedimientos, letras))
codificacion_reversa = dict(zip(letras, procedimientos))

torax_pandas = torax.to_pandas()
long_prestaciones = pd.melt(
    torax_pandas,
    id_vars=["ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=procedimientos,
)

long_prestaciones["variable"] = long_prestaciones["variable"].replace(codificacion)

In [55]:
por_procedimiento = (
    long_prestaciones.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "variable"])["value"]
    .value_counts()
    .reset_index(name="n_procedimientos")
)

por_procedimiento["variable"] = por_procedimiento["variable"].replace(codificacion_reversa)
por_procedimiento = por_procedimiento.rename({"value": "codigo_procedimiento"}, axis=1)


### Obtener diccionarios CIE

In [66]:
cie_9 = pd.read_excel('input/CIE-9.xlsx', dtype={'Código': str})

cie_10 = pd.read_excel('input/CIE-10.xlsx')

In [67]:
unido_proc = por_procedimiento.merge(cie_9, how='left', left_on='codigo_procedimiento',
                                            right_on='Código', suffixes=('', 'procedimientos'))

unido_diag = unido_proc.merge(cie_10, how='inner')

In [71]:
unido_proc

,ANIO_EGRESO,DIAGNOSTICO1,variable,codigo_procedimiento,n_procedimientos,Código,Descripción,Categoría,Sección,Capítulo
0,2019,A08.4,PROCEDIMIENTO1,89.54,1,89.54,MONITORIZACION ELECTROCARDIOGRAFICA,"89 ENTREVISTA, EVALUACION CONSULTA Y EXAMEN",16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
1,2019,A08.4,PROCEDIMIENTO2,38.91,1,38.91,CATETERISMO ARTERIA,"38 INCISION, EXCISION Y OCLUSION DE VASOS",07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...
2,2019,A08.4,PROCEDIMIENTO3,99.26,1,99.26,INYECCION TRANQUILIZANTE,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
3,2019,A08.4,PROCEDIMIENTO4,99.29,1,99.29,INYECCION SUSTANCIA TERAPEUTICA O PROFILACTICA...,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
4,2019,A08.4,PROCEDIMIENTO5,99.18,1,99.18,INYECCION ELECTROLITO,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
...,...,...,...,...,...,...,...,...,...,...
78011,2021,Z54.0,PROCEDIMIENTO3,93.96,1,93.96,ENRIQUECIMIENTO OXIGENO OTRO,"93 FISIOTERAPIA, TERAPIA RESPIRATORIA,REHABILI...",16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
78012,2021,Z54.0,PROCEDIMIENTO4,90.59,1,90.59,EXAMEN MICROSCOPICO SANGRE.OTRO,90 EXAMEN MICROSCOPICO I,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
78013,2021,Z54.0,PROCEDIMIENTO5,89.65,1,89.65,GASOMETRIA ARTERIA,"89 ENTREVISTA, EVALUACION CONSULTA Y EXAMEN",16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...
78014,2021,Z54.0,PROCEDIMIENTO6,93.99,1,93.99,TERAPIA RESPIRATORIA.OTRA,"93 FISIOTERAPIA, TERAPIA RESPIRATORIA,REHABILI...",16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...


In [29]:
por_procedimiento.to_excel("output/prestaciones_por_procedimiento.xlsx")